# Análisis y visualización de datos con Python
# 4. Limpieza de datos

    - a) Estrategia de limpieza
    - b) Valores faltantes y duplicados
    - c) Selección
    - d) Limpieza de texto
    - e) Limpieza de categóricos
    - f) Limpieza de números
    - g) Limpieza de fechas y tipos especiales
    - h) Generación de variables
    - h) Ordenar y guardar datos
    - i) Resumen

---


La **limpieza de datos** consiste en detectar y corregir errores, inconsistencias y valores atípicos. Este proceso es clave para asegurar la calidad de los análisis y la fiabilidad de los resultados. Los datos que se obtienen de fuentes del mundo real casi nunca están listos para ser utilizados directamente, ya que suelen contener errores de captura, formatos variados, o datos faltantes. Un análisis basado en datos sucios puede llevar a conclusiones incorrectas y a decisiones erróneas.

Para esta lección, utilizaremos el conjunto de datos de las Estadísticas de Defunciones Registradas (EDR) del INEGI, con el que ya hemos trabajado. El objetivo es aplicar las técnicas de limpieza de datos a este conjunto de información, con la meta de tener un conjunto de datos robusto para futuros análisis.

Los pasos que seguiremos en este notebook son:

* **Pregunta de investigación**: Definiremos una pregunta que guíe el proceso de limpieza.
* **Selección de subconjunto de interés**: Nos concentraremos en las variables que son relevantes para nuestra pregunta.
* **Manejo de valores faltantes y duplicados**: Identificaremos y corregiremos los datos ausentes o registros repetidos.
* **Mapeo de catálogos**: Transformaremos los códigos numéricos a etiquetas descriptivas para una mejor interpretación.
* **Revisión de tipos de datos**: Nos aseguraremos de que las variables estén en el formato correcto (ej. números, texto, fechas).
* **Ingeniería de variables**: Crearemos nuevas variables a partir de las existentes para enriquecer el análisis.
* **Guardar el conjunto de datos limpio**: Almacenaremos el resultado de nuestro trabajo.

In [1]:
from dbfread import DBF
import pandas as pd
import numpy as np

file_path = './data_raw/defunciones_base_datos_2023_dbf/DEFUN23.dbf'
df = DBF(file_path)
df = pd.DataFrame(df)
df.tail()

,ENT_REGIS,MUN_REGIS,TLOC_REGIS,LOC_REGIS,ENT_RESID,MUN_RESID,TLOC_RESID,LOC_RESID,ENT_OCURR,MUN_OCURR,...,COMPLICARO,DIA_CERT,MES_CERT,ANIO_CERT,MATERNAS,ENT_OCULES,MUN_OCULES,LOC_OCULES,RAZON_M,DIS_RE_OAX
799864,32,024,8,0001,32,024,8,0001,32,024,...,8,99,12,2023,,88,888,8888,NaN,999
799865,32,056,13,0001,32,056,13,0001,32,056,...,8,5,6,2023,,88,888,8888,NaN,999
799866,32,056,13,0001,32,017,4,0042,32,056,...,8,9,8,2023,,88,888,8888,NaN,999
799867,32,056,13,0001,32,017,13,0001,32,056,...,8,19,9,2023,,88,888,8888,NaN,999
799868,32,038,5,0001,32,038,1,0075,32,038,...,8,6,4,2023,,88,888,8888,NaN,999


## 4.a Pregunta de investigación

Un buen análisis de datos siempre comienza con **una pregunta clara**. 

Una pregunta de investigación clara en el análisis de datos tiene las siguientes características:
* **Específica**. Se enfoca en un problema o aspecto particular del conjunto de datos. En lugar de preguntar "¿Qué podemos aprender de los datos?", una pregunta específica sería "¿Cuál es la relación entre el sexo y la edad de las defunciones por accidentes?".
* **Relevante**. Está alineada con el objetivo del proyecto. La pregunta debe buscar una solución a un problema real o generar un nuevo conocimiento.
* **Viable**. Puede ser respondida con los datos y las herramientas disponibles. Una buena pregunta de investigación considera las limitaciones del conjunto de datos, como los datos faltantes o la falta de ciertas variables.
* **Pregunta una relación** Generalmente, busca una relación, un patrón, o una causa-efecto entre diferentes variables, no una simple descripción. En lugar de preguntar "¿Cuántas defunciones hay?", una pregunta de investigación sería "¿Hay una correlación entre las defunciones y la estación del año?".
* **No es ambigua**. La pregunta debe ser formulada de tal manera que no dé lugar a múltiples interpretaciones. El resultado del análisis debería proporcionar una respuesta clara, incluso si es una respuesta negativa.

Una estrategia útil es recordar las preguntas clave: qué, quién, cómo, cuándo, dónde y por qué.

Por ejemplo, trabajaremos las **muertes por causas respiratorias**. Una estrategia para acotar la pregunta es recordar las preguntas clave: qué, quién, cómo, cuándo, dónde y por qué.

* ¿Qué y por qué?: comportamiento de las defunciones por causas respiratorias
* ¿Quiénes son?: grupo etario (sexo y edad) de las personas fallecidas.
* ¿Cuándo y dónde?: tiempo y el lugar de las defunciones durante 2023 en México.
* ¿Cómo se atendieron?: sistema de salud al que pertenecían.

A partir de eso nuestra pregunta es:
**¿Cómo se comportaron las defunciones por causas respiratorias en México durante el año 2023, y cuál fue el perfil demográfico y de atención médica de las personas afectadas?**

### Estrategia de selección y limpieza de datos

Con base en los hallazgos de nuestro análisis exploratorio, hemos desarrollado una estrategia de selección y limpieza de datos. Estos pasos nos permitirán obtener un conjunto de datos confiable para responder a nuestra pregunta de investigación.

1. **Selección de variables**: Trabajaremos exclusivamente con las variables que son relevantes para el análisis.
2. **Filtrado por año**: Solo conservaremos las defunciones ocurridas en el año 2023. Esto nos ayudará a evitar datos de años atípicos que se registraron tardíamente.
3. **Filtrado de causas de muerte**: Ampliaremos la selección de causas para incluir no solo los códigos que inician con `'J'`, sino también otros códigos relevantes de la CIE-10 que representan enfermedades respiratorias. Por ejemplo, incluiremos códigos de tuberculosis como `'A15'`, `'A16'`, `'A19'` y `'B90.9'`, que pertenecen a otros capítulos.
4. **Manejo de valores faltantes y no especificados**: Los datos del INEGI no usan `NaN` para los valores ausentes, sino códigos como `999`, `9999` o espacios en blanco. Corregiremos estas codificaciones para que Python las reconozca como valores faltantes.
5. **Mapeo de catálogos**: Transformaremos los códigos numéricos de columnas como `SEXO` y `DERECHOHAB` a valores descriptivos más comprensibles. También consideraremos las variables `EDAD`, `ENT_OCURR` y `MUN_OCURR`, ya que su formato requiere una conversión a años para un análisis correcto.
* **Revisión y conversión de tipos de datos**: Nos aseguraremos de que cada columna tenga el tipo de dato correcto. Por ejemplo, las variables de fecha (`'DIA_OCURR'`, `'MES_OCURR'`, `'ANIO_OCUR'`) se convertirán a un único tipo `datetime`, lo que permitirá realizar análisis de tiempo.
* **Ingeniería de variables**: Una vez que las columnas de fecha estén unificadas, crearemos nuevas variables, como el día de la semana, que pueden ser útiles para identificar patrones temporales en la mortalidad.
* **Guardar el conjunto de datos limpio**: Finalmente, exportaremos el DataFrame, esto nos permitirá cargar directamente un conjunto de datos limpio en futuros análisis.

## 4.b Selección de datos

A partir de nuestras preguntas y los errores detectados, repetiremos el proceso de selección de datos incorporando las correcciones necesarias. Para validar el proceso nos fijaremos en el tamaño del DataFrame y a final haremos una revisión.

Primero, seleccionaremoslas variables, o columnas, que son relevantes para el análisis.

In [2]:
# Seleccionar las columnas de interés
columnas = ['SEXO', 'EDAD', 'EDAD_AGRU', 'ENT_OCURR', 'MUN_OCURR', 'AREA_UR', 'DIA_OCURR', 'MES_OCURR', 
            'ANIO_OCUR', 'CAUSA_DEF', 'TIPO_DEFUN', 'SITIO_OCUR', 'COND_CERT', 'DERECHOHAB']
df_respiratorio = df[columnas]

# Imprimir tamaño resultante
print(f"Filas df filtrado: {df_respiratorio.shape[0]}/{df.shape[0]}")
print(f"Colum df filtrado: {df_respiratorio.shape[1]}/{df.shape[1]}")

Filas df filtrado: 799869/799869
Colum df filtrado: 14/74


Después, seleccionaremos las observaciones, o filas, que corresponden a defunciones ocurridas en el año 2023. Esto nos ayudará a evitar datos de años atípicos que se registraron tardíamente.

In [3]:
# Filtrar defunciones ocurridas en 2023
df_respiratorio = df_respiratorio[df_respiratorio['ANIO_OCUR']==2023]

# Imprimir tamaño resultante
print(f"Filas df filtrado: {df_respiratorio.shape[0]}/{df.shape[0]}")
print(f"Colum df filtrado: {df_respiratorio.shape[1]}/{df.shape[1]}")

Filas df filtrado: 779239/799869
Colum df filtrado: 14/74


Para obtener una selección precisa de las defunciones por causas respiratorias, nos basaremos en una lista de códigos de la **Clasificación Internacional de Enfermedades (CIE-10)**, que fue tomada de la investigación de Pérez-Padilla (2023). Esta lista, que se encuentra en el archivo `codigoCIE_respiratoria.csv`, contiene los códigos, el término asociado y una clasificación general de las enfermedades de interés.

En este caso solo necesitamos los datos de tres columnas: `icd_code`, `inegi_name`, `class_code`.

In [4]:
# Carga el catálogo de muertes respiratorias
cat_respiratorio = pd.read_csv(
                        'data_raw/codigoCIE_respiratoria.csv',
                        usecols=['icd_code', 'class_code']
                        )
cat_respiratorio

,icd_code,class_code
0,R84,Other respiratory disorders
1,R06,Other respiratory disorders
2,J21,Acute bronchitis and bronchiolitis
3,J20,Acute bronchitis and bronchiolitis
4,J22X,Acute lower respiratory infection
...,...,...
228,A19,TB and complications
229,B909,TB and complications
230,F17,Use of tobacco and other drugs
231,F18,Use of tobacco and other drugs


La clave de este proceso es entender la estructura jerárquica de los códigos CIE-10. Los códigos pueden tener tres o cuatro caracteres. Por ejemplo, el código `J20` representa la "Bronquitis aguda", mientras que el código `J20.6` es "Bronquitis aguda por rinovirus", una subcategoría más específica. La tabla de defunciones del INEGI utiliza códigos de cuatro caracteres, pero nuestro catálogo de referencia puede contener códigos de tres, lo que implica que todos los códigos de cuatro caracteres que inician con esos tres dígitos deben ser incluidos.

Para manejar esta particularidad, utilizaremos la función `map_cie10_diagnostics_generalized`, una herramienta que se ha diseñado específicamente para este problema. Si quieres saber más sobre cómo funciona esta función y de cómo usar IA generativa para hacer funciones personalizadas, ve el video: [Vibe codding: filtrando códigos del CIE10](url).

Primero es necesario asegurarnos de que la función esté cargada en la memoria de trabajo, para después aplicarla a los datos.

In [5]:
def map_cie10_diagnostics_generalized(df, catalog, df_cie_col, catalog_cie_col):
    """
    Maps CIE-10 diagnostics from a dataframe to a catalog, handling both 3- and 4-digit codes.

    Args:
        df (pd.DataFrame): The main dataframe.
        catalog (pd.DataFrame): The catalog dataframe.
        df_cie_col (str): The name of the column in `df` with CIE-10 codes (4 digits).
        catalog_cie_col (str): The name of the column in `catalog` with CIE-10 codes (3 or 4 digits).

    Returns:
        pd.DataFrame: The original dataframe with the additional columns from the
                      catalog joined.
    """
    # Create a 3-digit code column in the catalog for broader matching
    catalog_3digit_col = f"{catalog_cie_col}_3digit"
    catalog[catalog_3digit_col] = catalog[catalog_cie_col].str[:3]

    # First merge: Attempt a direct match on the full 4-digit codes.
    df_mapped = pd.merge(df, catalog, left_on=df_cie_col, right_on=catalog_cie_col, how='left')

    # Find the rows that did not match in the first merge
    unmatched_df = df_mapped.loc[ df_mapped[catalog_cie_col].isnull() ]
    matched_df = df_mapped.loc[ df_mapped[catalog_cie_col].notnull() ]

    if not unmatched_df.empty:
        # Create a 3-digit code from the main dataframe for the second merge
        df_3digit_col = f"{df_cie_col}_3digit"
        unmatched_df[df_3digit_col] = unmatched_df[df_cie_col].str[:3]

        # Merge the unmatched rows on the 3-digit codes.
        merged_unmatched = pd.merge(
            unmatched_df.drop(columns=[catalog_cie_col, catalog_3digit_col]),
            catalog,
            left_on=df_3digit_col,
            right_on=catalog_3digit_col,
            how='left',
            suffixes=('_orig', '_new')
        )

        # Combine the successfully matched rows with the newly matched rows.
        final_df = pd.concat([matched_df, merged_unmatched])
        final_df = final_df.loc[final_df[catalog_cie_col].notna()]

        # Join 3 and 4 digit merges and clean up temporary columns.
        final_df = final_df.drop(columns=[df_3digit_col, catalog_3digit_col])
        for col in catalog.drop(columns=[catalog_cie_col, catalog_3digit_col]).columns:
            final_df[col] = final_df[col].fillna(final_df[f"{col}_orig"]).fillna(final_df[f"{col}_new"])
            final_df = final_df.drop(columns=[f"{col}_orig", f"{col}_new"])
            
    else:
        final_df = df_mapped
        final_df = final_df.drop(columns=[catalog_3digit_col]).reset_index(drop=True)

    return final_df

# Aplica la función personalizada para filtrar el DataFrame principal
# La función devuelve un nuevo DataFrame con solo las causas de muerte de interés
df_respiratorio = map_cie10_diagnostics_generalized(
                        df=df_respiratorio,
                        catalog=cat_respiratorio,
                        df_cie_col='CAUSA_DEF',
                        catalog_cie_col='icd_code'
                        )
df_respiratorio

# Imprimir tamaño resultante
print(f"Filas df filtrado: {df_respiratorio.shape[0]}/{df.shape[0]}")
print(f"Colum df filtrado: {df_respiratorio.shape[1]}/{df.shape[1]}")

/tmp/ipykernel_23146/346410664.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unmatched_df[df_3digit_col] = unmatched_df[df_cie_col].str[:3]


Filas df filtrado: 104334/799869
Colum df filtrado: 16/74


Al revisar este nuevo reporte, notarás que el número de filas es menor, confirmando que el filtrado se aplicó correctamente. 
Además, las nuevas columnas del catálogo estarán presentes. Podemos quitar la columna de `icd_code`, que ya no es de utilidad. Además, cambiaremos el nombre de la columna `class_code` por algo más significativo usando la función `.rename()`

La función `rename()` cambia el nombre de las etiquetas de fila (índice) o de las etiquetas de columna. La sintaxis básica para renombrar una columna es: `df.rename(columns={'nombre_antiguo': 'nombre_nuevo'})`.

Nota: Para asegurarte de que el filtro funcionó como esperas y que las nuevas columnas del catálogo se añadieron correctamente, puedes realizar un nuevo perfil de datos.

In [6]:
df_respiratorio = df_respiratorio.drop(columns='icd_code') \
                    .rename(columns={'class_code':'CAUSA_DEF_CLAS'})
df_respiratorio.columns

Index(['SEXO', 'EDAD', 'EDAD_AGRU', 'ENT_OCURR', 'MUN_OCURR', 'AREA_UR',
       'DIA_OCURR', 'MES_OCURR', 'ANIO_OCUR', 'CAUSA_DEF', 'TIPO_DEFUN',
       'SITIO_OCUR', 'COND_CERT', 'DERECHOHAB', 'CAUSA_DEF_CLAS'],
      dtype='object')

## 4.c Manejo de valores faltantes y no especificados

Los datos del INEGI tienen una particularidad: no usan el estándar `NaN` de Python para los valores ausentes, sino que los codifican con valores numéricos específicos, como `999`, `9999` o incluso con cadenas de texto en blanco. Para que nuestro análisis sea preciso, es necesario corregir esta codificación y asegurar que estos valores se traten como `NaN`, lo que permite que las funciones de pandas los ignoren adecuadamente.

Para entender cómo el INEGI codifica esta información, es necesario consultar los catálogos que acompañan a la base de datos. Por ejemplo:

  * En la columna **`SEXO`**, el valor `9` se usa para indicar un sexo "No especificado".
  * En la columna **`DIA_OCURR`**, el valor `99` significa "No especificado", mientras que `9` es un día válido.

Esta distinción es fundamental y subraya por qué la exploración de metadatos es una etapa tan importante.

Para nuestro caso, decidiremos sustituir los valores de no especificado solo en las columnas de fecha (`'DIA_OCURR'`, `'MES_OCURR'`, `'ANIO_OCUR'`) por `NaN`, ya que estos códigos podrían causar un error de tipo o un análisis incorrecto al momento de generar la variable `datetime`.

Para esto, usaremos la función `.replace()`, la cual busca un valor o conjunto de valores específicos en una Serie o un DataFrame y los sustituye por otro valor.

Sintaxis básica: `df['columna'].replace(valores_a_cambiar, nuevo_valor)`.

In [7]:
# Reemplazar valores no especificados por NaN
df_respiratorio['DIA_OCURR'] = df_respiratorio['DIA_OCURR'].replace(99, np.nan)
df_respiratorio['MES_OCURR'] = df_respiratorio['MES_OCURR'].replace(99, np.nan)
df_respiratorio['ANIO_OCUR'] = df_respiratorio['ANIO_OCUR'].replace(9999, np.nan)

# Verificar los cambios
cols = ['DIA_OCURR', 'MES_OCURR', 'ANIO_OCUR']
df_respiratorio.loc[df_respiratorio[cols].isna().any(axis=1), cols]

,DIA_OCURR,MES_OCURR,ANIO_OCUR
9531,NaN,1.0,2023
118593,NaN,NaN,2023
59070,NaN,NaN,2023
110954,NaN,NaN,2023
180487,NaN,NaN,2023
185220,NaN,NaN,2023
205107,NaN,NaN,2023
288732,NaN,NaN,2023
301993,NaN,NaN,2023
331556,NaN,NaN,2023


## 4.d Mapeo de catálogos

Para que los datos sean más comprensibles, es necesario convertir los códigos numéricos a valores descriptivos. Usaremos el método `.replace()` de pandas, el cual también funciona con diccionarios para mapear valores. Si un valor en el DataFrame no se encuentra en el diccionario, se mantendrá sin cambios.

Empezaremos con la columna `'SEXO'`. Su catálogo es el siguiente:

  * `1`: Hombre
  * `2`: Mujer
  * `9`: No especificado

In [8]:
# Definir el diccionario de mapeo para la columna SEXO
mapeo_sexo = { 1: 'Hombre', 2: 'Mujer', 9: 'No especificado' }

# Aplicar el reemplazo en la columna 'SEXO'
df_respiratorio['SEXO'] = df_respiratorio['SEXO'].replace(mapeo_sexo)

# Mostrar los valores únicos para verificar el cambio
df_respiratorio['SEXO'].value_counts(dropna=False)

SEXO
Hombre             61950
Mujer              42368
No especificado       16
Name: count, dtype: int64

Ahora es necesario repetir este proceso para el resto de las variables a mapear:

In [9]:
# AREA_UR
mapeo_area_ur = { 1: "Urbana", 2: "Rural", 9: "No especificada" }
df_respiratorio['AREA_UR'] = df_respiratorio['AREA_UR'].replace(mapeo_area_ur)

# TIPO_DEFUN
mapeo_tipo_defun = { 1: "Accidente", 2: "Homicidio (Agresión)", 3: "Suicidio (Lesión autoinfligida)", 
                     4: "Enfermedad (Muerte natural)", 5: "Intervención legal", 9: "Se ignora" }
df_respiratorio['TIPO_DEFUN'] = df_respiratorio['TIPO_DEFUN'].replace(mapeo_tipo_defun)

# SITIO_OCUR
mapeo_sitio_ocur = { 1: "Secretaría de Salud", 2: "IMSS BIENESTAR", 3: "IMSS", 4: "ISSSTE", 5: "PEMEX", 
                     6: "Secretaría de la Defensa Nacional (SEDENA)", 7: "Secretaría de Marina (SEMAR)",
                     8: "Otra unidad pública", 9: "Unidad médica privada", 10: "Vía pública", 11: "Hogar",
                     12: "Otro lugar", 99: "No especificado"  }
df_respiratorio['SITIO_OCUR'] = df_respiratorio['SITIO_OCUR'].replace(mapeo_sitio_ocur)

# COND_CERT
mapeo_cond_cert = { 1: "Médico tratante", 2: "Médico legista", 3: "Otro médico", 4: "Persona autorizada por la SSA", 
                    5: "Autoridad civil", 8: "Otra", 9: "No especificada"  }
df_respiratorio['COND_CERT'] = df_respiratorio['COND_CERT'].replace(mapeo_cond_cert)

# DERECHOHAB
mapeo_derechohab = { 1: "Ninguna", 2: "IMSS", 3: "ISSSTE", 4: "PEMEX", 5: "SEDENA", 6: "SEMAR", 7: 
                     "Seguro Popular", 8: "Otra", 9: "IMSS BIENESTAR", 10: "ISSFAM", 99: "No especificada"  }
df_respiratorio['DERECHOHAB'] = df_respiratorio['DERECHOHAB'].replace(mapeo_derechohab)

# Mostrar los valores para verificar el cambio
df_respiratorio.tail()

,SEXO,EDAD,EDAD_AGRU,ENT_OCURR,MUN_OCURR,AREA_UR,DIA_OCURR,MES_OCURR,ANIO_OCUR,CAUSA_DEF,TIPO_DEFUN,SITIO_OCUR,COND_CERT,DERECHOHAB,CAUSA_DEF_CLAS
775307,Hombre,1097,01,32,056,Rural,11.0,3.0,2023,P251,Enfermedad (Muerte natural),IMSS,Médico tratante,IMSS,"Neonatal hypoxia, aspiration, neonatal pneumonia"
775313,Hombre,1012,01,32,055,Rural,19.0,6.0,2023,P209,Enfermedad (Muerte natural),IMSS BIENESTAR,Médico tratante,Ninguna,"Neonatal hypoxia, aspiration, neonatal pneumonia"
775315,Mujer,2021,01,32,024,Urbana,22.0,12.0,2023,P249,Enfermedad (Muerte natural),Hogar,Otro médico,No especificada,"Neonatal hypoxia, aspiration, neonatal pneumonia"
775317,Mujer,1003,01,32,056,Urbana,9.0,8.0,2023,P220,Enfermedad (Muerte natural),IMSS,Médico tratante,IMSS,"Neonatal hypoxia, aspiration, neonatal pneumonia"
775319,Hombre,1023,01,32,038,Rural,5.0,4.0,2023,P285,Enfermedad (Muerte natural),IMSS BIENESTAR,Otro médico,Ninguna,"Neonatal hypoxia, aspiration, neonatal pneumonia"


### Mapeo de causas de defunción

Las causas de defunción, identificadas por los códigos de la CIE-10, se encuentran en el archivo `CATMINDE.dbf`. El objetivo es usar este catálogo para añadir los nombres de las enfermedades a nuestro DataFrame. A diferencia de las variables anteriores que se mapearon con `replace()`, en este caso, es más eficiente usar una función de combinación de datos como `merge()`.

`merge()` es una función que combina dos DataFrames basándose en valores comunes en una o más columnas, de manera similar a como se unen tablas en bases de datos. Veremos esta función en detalle en el siguiente notebook.

Fijate cómo hemos cambiado los nombres de las columnas.

In [10]:
file_path = './data_raw/defunciones_base_datos_2023_dbf/CATMINDE.dbf'
mapeo_causa = DBF(file_path)
mapeo_causa = pd.DataFrame(mapeo_causa)
mapeo_causa.columns = ['CAUSA_DEF', 'CAUSA_DEF_NOM']
mapeo_causa.head()

,CAUSA_DEF,CAUSA_DEF_NOM
0,A010,Fiebre tifoidea
1,A014,"Fiebre paratifoidea, no especificada"
2,A020,Enteritis debida a Salmonella
3,A022,Infecciones localizadas debidas a Salmonella
4,A028,Otras infecciones especificadas como debidas a...


Ahora uniremos el DataFrame de defunciones al DataFrame de causas con `.merge()`.

In [11]:
# Realizar el merge para añadir la columna 'CAUSA_DEF_NOM'
df_respiratorio = pd.merge( df_respiratorio, mapeo_causa, on='CAUSA_DEF', how='left' )

#Revisar
df_respiratorio[['CAUSA_DEF', 'CAUSA_DEF_NOM', 'CAUSA_DEF_CLAS']].tail()

,CAUSA_DEF,CAUSA_DEF_NOM,CAUSA_DEF_CLAS
104329,P251,Neumotórax originado en el período perinatal,"Neonatal hypoxia, aspiration, neonatal pneumonia"
104330,P209,"Hipoxia intrauterina, no especificada","Neonatal hypoxia, aspiration, neonatal pneumonia"
104331,P249,"Síndrome de aspiración neonatal, sin otra espe...","Neonatal hypoxia, aspiration, neonatal pneumonia"
104332,P220,Síndrome de dificultad respiratoria del recién...,"Neonatal hypoxia, aspiration, neonatal pneumonia"
104333,P285,Insuficiencia respiratoria del recién nacido,"Neonatal hypoxia, aspiration, neonatal pneumonia"


### Mapeo de catálogos geográficos

Para interpretar las variables de ubicación, `ENT_OCURR` y `MUN_OCURR`, es necesario convertirlas de códigos a nombres. La información para este mapeo se encuentra en el archivo `CATEMLDE23.dbf`, que contiene los catálogos de entidades, municipios y localidades.

In [12]:
file_path = './data_raw/defunciones_base_datos_2023_dbf/CATEMLDE23.dbf'
mapeo_lugar = DBF(file_path)
mapeo_lugar = pd.DataFrame(mapeo_lugar)
mapeo_lugar.head()

,CVE_ENT,CVE_MUN,CVE_LOC,NOM_LOC
0,01,000,0000,Aguascalientes
1,01,001,0000,Aguascalientes
2,01,001,0001,Aguascalientes
3,01,001,0121,Cabecita 3 Marías (Rancho Nuevo)
4,01,001,0127,Los Caños


La primera tarea es cargar el archivo del catálogo y prepararlo para el mapeo. Es importante notar que el catálogo incluye códigos especiales (`CVE_MUN` con `000` o `CVE_LOC` con `0000`) que representan el total de una entidad o un municipio. 

Además. el catálogo es **jerárquico**. Esto significa que las claves del municipio (`CVE_MUN`) dependen de la clave de la entidad (`CVE_ENT`). Por lo tanto, no se puede mapear un municipio solo con su código, ya que podría estar repetido en diferentes entidades. Para lograr esto usaremos la función `.merge()`, cuya explicación veremos más adelante.

Primero, mapearemos las entidades:

In [14]:
# Seleccionar solo las entidades y columnas de interes
mapeo_ent = mapeo_lugar.loc[mapeo_lugar['CVE_MUN']=='000', ['CVE_ENT','NOM_LOC']]
# cambiar nombre columnas
mapeo_ent.columns = ['ENT_OCURR', 'ENT_OCURR_NOM']
display( mapeo_ent.tail() )

# Unir ambas tablas
df_respiratorio = pd.merge( df_respiratorio, mapeo_ent, on='ENT_OCURR', how='left' )
#Revisar
df_respiratorio[['ENT_OCURR', 'MUN_OCURR', 'ENT_OCURR_NOM']].tail()

,ENT_OCURR,ENT_OCURR_NOM
27004,31,Yucatán
27559,32,Zacatecas
28274,33,Estados Unidos de Norteamérica
28277,88,Entidad no aplica para A00 - R99 Y V90 - Y89
28280,99,Entidad no especificada


,ENT_OCURR,MUN_OCURR,ENT_OCURR_NOM
104329,32,056,Zacatecas
104330,32,055,Zacatecas
104331,32,024,Zacatecas
104332,32,056,Zacatecas
104333,32,038,Zacatecas


Ahora seleccionaremos los nombres de municipios, es decir, aquellas filas del catálogo de lugares donde `CVE_LOC` es  `'0000'`. A continuación uniremos las dos tablas con `.merge()`. Cómo el catálogo es jerárquico es necesario tomar en cuenta tanto la columna de entidades (`CVE_ENT`) como la de municipios (`CVE_MUN`)

In [16]:
# Seleccionar solo las entidades y columnas de interes
mapeo_mun = mapeo_lugar.loc[mapeo_lugar['CVE_LOC']=='0000', ['CVE_ENT', 'CVE_MUN','NOM_LOC']]
# cambiar nombre columnas
mapeo_mun.columns = ['ENT_OCURR', 'MUN_OCURR', 'MUN_OCURR_NOM']
display( mapeo_mun.tail() )

# Unir ambas tablas
df_respiratorio = pd.merge( df_respiratorio, mapeo_mun, on=['ENT_OCURR', 'MUN_OCURR'], how='left' )
#Revisar
df_respiratorio[['ENT_OCURR', 'MUN_OCURR', 'ENT_OCURR_NOM', 'MUN_OCURR_NOM']].tail()

,ENT_OCURR,MUN_OCURR,MUN_OCURR_NOM
28275,33,999,Municipio no especificado
28277,88,000,Entidad no aplica para A00 - R99 Y V90 - Y89
28278,88,888,Municipio no aplica para A00 - R99 Y V90 - Y89
28280,99,000,Entidad no especificada
28281,99,999,Municipio no especificado


,ENT_OCURR,MUN_OCURR,ENT_OCURR_NOM,MUN_OCURR_NOM
104329,32,056,Zacatecas,Zacatecas
104330,32,055,Zacatecas,Villanueva
104331,32,024,Zacatecas,Loreto
104332,32,056,Zacatecas,Zacatecas
104333,32,038,Zacatecas,Pinos


### Mapeo de catálogos de edad

La variable `EDAD` utiliza un código numérico para representar la edad en diferentes escalas (días, meses, años). Para nuestro análisis, nos interesa la edad en años. Convertiremos los códigos de edades menores a un año a `0` y los códigos de años no especificados a `NaN` para facilitar los cálculos numéricos.

Para aplicar una lógica de conversión compleja a cada fila de una columna, crearemos una función especial `modify_age_inegi` y la aplicaremos a la columna `Edad` usando la función `apply()`.

La función `apply()` es una forma flexible de aplicar una función a cada elemento de una serie o a lo largo de los ejes de un DataFrame. Su sintaxis básica es `df[columna].apply(nombre_de_la_funcion)`. Es útil para operaciones personalizadas que no están disponibles en las funciones predefinidas de pandas.

A continuación, la función `modify_age_inegi` convierte los códigos de edad del INEGI:

  * Si el código es `4998`, que representa la edad no especificada, devuelve un valor nulo (`np.nan`).
  * Si el código es menor a `4000`, que corresponde a edades en días o meses, devuelve `0`.
  * Para los códigos que comienzan con `4` (edades en años), resta `4000` para obtener la edad real.

In [20]:
def modify_age_inegi(number):
    if number == 4998:
        return np.nan
    elif number < 4000:
        return 0
    else:
        return number - 4000

# Aplicar la función a la columna 'EDAD' y crear una nueva columna 'EDAD_ANOS'
df_respiratorio['EDAD'] = df_respiratorio['EDAD'].apply(modify_age_inegi)

# Mostrar los valores únicos de la nueva columna para verificar el cambio
df_respiratorio['EDAD'].value_counts().sort_index()

EDAD
0.0      6739
1.0       606
2.0       289
3.0       210
4.0       145
         ... 
110.0       1
112.0       6
114.0       2
119.0       1
120.0       1
Name: count, Length: 115, dtype: int64

La columna `EDAD_AGRU` clasifica las defunciones en rangos de edad predefinidos, lo cual es útil para análisis demográficos. Para hacer esta variable más legible, la mapearemos de códigos numéricos a descripciones textuales.

A continuación, se muestra el diccionario que utilizaremos para la conversión. Este catálogo está **ordenado** de forma natural de menor a mayor edad. Al hacer la conversión se pierde el orden implícito en la numeración, arreglaremos esto más adelante.

In [24]:
mapeo_edad_agru = { 
                    '01':'Menores de un año', '02':'De un año', '03':'De 2', '04':'De 3', '05':'De 4', '06':'De 5 a 9', 
                    '07':'De 10 a 14', '08':'De 15 a 19', '09':'De 20 a 24', '10':'De 25 a 29', '11':'De 30 a 34', 
                    '12':'De 35 a 39', '13':'De 40 a 44', '14':'De 45 a 49', '15':'De 50 a 54', '16':'De 55 a 59', 
                    '17':'De 60 a 64', '18':'De 65 a 69', '19':'De 70 a 74', '20':'De 75 a 79', '21':'De 80 a 84', 
                    '22':'De 85 a 89', '23':'De 90 a 94', '24':'De 95 a 99', '25':'De 100 a 104', '26':'De 105 a 109', 
                    '27':'De 110 a 114', '28':'De 115 a 119', '29':'De 120 y más', '30':'No especificada' 
                  }

# Aplicar el reemplazo en la columna 'EDAD_AGRU'
df_respiratorio['EDAD_AGRU'] = df_respiratorio['EDAD_AGRU'].replace(mapeo_edad_agru)

# Mostrar el conteo de valores únicos para verificar el cambio
df_respiratorio['EDAD_AGRU'].value_counts()

EDAD_AGRU
De 10 a 14             398
De 100 a 104           519
De 105 a 109            48
De 110 a 114             9
De 115 a 119             1
De 120 y más             1
De 15 a 19             851
De 2                   289
De 20 a 24            1667
De 25 a 29            2523
De 3                   210
De 30 a 34            3221
De 35 a 39            3224
De 4                   145
De 40 a 44            3521
De 45 a 49            3805
De 5 a 9               374
De 50 a 54            4454
De 55 a 59            5351
De 60 a 64            6625
De 65 a 69            8025
De 70 a 74            9409
De 75 a 79           10933
De 80 a 84           11383
De 85 a 89           10394
De 90 a 94            6738
De 95 a 99            2614
De un año              606
Menores de un año     6739
No especificada        257
Name: count, dtype: int64

Con este paso, hemos completado el mapeo de las variables de interés.

In [25]:
df_respiratorio.columns

Index(['SEXO', 'EDAD', 'EDAD_AGRU', 'ENT_OCURR', 'MUN_OCURR', 'AREA_UR',
       'DIA_OCURR', 'MES_OCURR', 'ANIO_OCUR', 'CAUSA_DEF', 'TIPO_DEFUN',
       'SITIO_OCUR', 'COND_CERT', 'DERECHOHAB', 'CAUSA_DEF_CLAS',
       'CAUSA_DEF_NOM', 'ENT_OCURR_NOM', 'MUN_OCURR_NOM'],
      dtype='object')

In [27]:
df_respiratorio.tail()

,SEXO,EDAD,EDAD_AGRU,ENT_OCURR,MUN_OCURR,AREA_UR,DIA_OCURR,MES_OCURR,ANIO_OCUR,CAUSA_DEF,TIPO_DEFUN,SITIO_OCUR,COND_CERT,DERECHOHAB,CAUSA_DEF_CLAS,CAUSA_DEF_NOM,ENT_OCURR_NOM,MUN_OCURR_NOM
104329,Hombre,0.0,Menores de un año,32,056,Rural,11.0,3.0,2023,P251,Enfermedad (Muerte natural),IMSS,Médico tratante,IMSS,"Neonatal hypoxia, aspiration, neonatal pneumonia",Neumotórax originado en el período perinatal,Zacatecas,Zacatecas
104330,Hombre,0.0,Menores de un año,32,055,Rural,19.0,6.0,2023,P209,Enfermedad (Muerte natural),IMSS BIENESTAR,Médico tratante,Ninguna,"Neonatal hypoxia, aspiration, neonatal pneumonia","Hipoxia intrauterina, no especificada",Zacatecas,Villanueva
104331,Mujer,0.0,Menores de un año,32,024,Urbana,22.0,12.0,2023,P249,Enfermedad (Muerte natural),Hogar,Otro médico,No especificada,"Neonatal hypoxia, aspiration, neonatal pneumonia","Síndrome de aspiración neonatal, sin otra espe...",Zacatecas,Loreto
104332,Mujer,0.0,Menores de un año,32,056,Urbana,9.0,8.0,2023,P220,Enfermedad (Muerte natural),IMSS,Médico tratante,IMSS,"Neonatal hypoxia, aspiration, neonatal pneumonia",Síndrome de dificultad respiratoria del recién...,Zacatecas,Zacatecas
104333,Hombre,0.0,Menores de un año,32,038,Rural,5.0,4.0,2023,P285,Enfermedad (Muerte natural),IMSS BIENESTAR,Otro médico,Ninguna,"Neonatal hypoxia, aspiration, neonatal pneumonia",Insuficiencia respiratoria del recién nacido,Zacatecas,Pinos


## 4.e Revisión y conversión de tipos de datos

Un paso clave en la limpieza de datos es asegurar que cada columna tenga el tipo de dato adecuado para su contenido. Aunque Pandas a menudo infiere los tipos de datos correctamente, a veces una variable numérica se lee como texto (object) o viceversa, lo que puede causar errores en el análisis.

Revisemos los tipos de datos actuales en nuestro DataFrame:

In [29]:
df_respiratorio.dtypes

SEXO               object
EDAD              float64
EDAD_AGRU          object
ENT_OCURR          object
MUN_OCURR          object
AREA_UR            object
DIA_OCURR         float64
MES_OCURR         float64
ANIO_OCUR           int64
CAUSA_DEF          object
TIPO_DEFUN         object
SITIO_OCUR         object
COND_CERT          object
DERECHOHAB         object
CAUSA_DEF_CLAS     object
CAUSA_DEF_NOM      object
ENT_OCURR_NOM      object
MUN_OCURR_NOM      object
dtype: object

Varias columnas categóricas se han leído como tipo `object` (cadena de texto). Si bien esto funciona, convertirlas explícitamente al tipo `category` tiene varias ventajas:

* Optimización de memoria: El tipo `category` almacena las variables como números enteros, lo que reduce el consumo de memoria, un aspecto útil al trabajar con grandes conjuntos de datos.
* Velocidad: Las operaciones de filtrado y agrupamiento son más rápidas.
* Claridad: Indica la intención del análisis, ya que diferencia entre variables nominales y textuales.

Para convertir una columna a tipo `category`, usamos el método `.astype()`.

In [31]:
# Convertir la columna 'SEXO' a tipo 'category'
df_respiratorio['SEXO'] = df_respiratorio['SEXO'].astype('category')

# Verificar el cambio
df_respiratorio['SEXO']

0         Hombre
1         Hombre
2         Hombre
3         Hombre
4         Hombre
           ...  
104329    Hombre
104330    Hombre
104331     Mujer
104332     Mujer
104333    Hombre
Name: SEXO, Length: 104334, dtype: category
Categories (3, object): ['Hombre', 'Mujer', 'No especificado']

Puedes notar que al final de la celda se muestra el tipo y las categorías.

Ahora, repitamos este proceso para todas las demás columnas categóricas.

In [34]:
# Usaremos un for para aplicar la misma operación a todas las columnas categóricas
for col in ['ENT_OCURR', 'MUN_OCURR', 'AREA_UR', 'CAUSA_DEF', 'TIPO_DEFUN', 
            'SITIO_OCUR', 'COND_CERT', 'DERECHOHAB', 'CAUSA_DEF_CLAS', 
            'CAUSA_DEF_NOM', 'ENT_OCURR_NOM', 'MUN_OCURR_NOM']:
    df_respiratorio[col] = df_respiratorio[col].astype('category')
# Mostrar el tipo de dato para verificar el cambio
df_respiratorio.dtypes

SEXO              category
EDAD               float64
EDAD_AGRU           object
ENT_OCURR         category
MUN_OCURR         category
AREA_UR           category
DIA_OCURR          float64
MES_OCURR          float64
ANIO_OCUR            int64
CAUSA_DEF         category
TIPO_DEFUN        category
SITIO_OCUR        category
COND_CERT         category
DERECHOHAB        category
CAUSA_DEF_CLAS    category
CAUSA_DEF_NOM     category
ENT_OCURR_NOM     category
MUN_OCURR_NOM     category
dtype: object

### Categorías ordenadas

Algunas variables categóricas, como los grupos de edad, tienen un orden inherente que es importante conservar para el análisis. Pandas permite manejar esto con el tipo de dato `Categorical`, que almacena el orden de los valores.

La columna `EDAD_AGRU` ya se ha mapeado a nombres de grupos de edad, pero su tipo de dato aún es `object`. Convertiremos esta columna a un tipo de categoría ordenado. Para hacerlo, usaremos la lista de categorías del diccionario de mapeo que creamos anteriormente para definir el orden.

Nota como al ordenar el conteo de valores ordena usando el orden definido por la categoría, en lugar de hacerlo alfabéticamente.

In [41]:
# Obtener la lista de categorías en el orden correcto
ordered_categories  = list(mapeo_edad_agru.values())
print(ordered_categories)

# Convertir la columna a un tipo de categoría ordenado
df_respiratorio['EDAD_AGRU'] = pd.Categorical(
    df_respiratorio['EDAD_AGRU'],
    categories=ordered_categories,
    ordered=True
)

# Mostrar el conteo de valores únicos y su orden
df_respiratorio['EDAD_AGRU'].value_counts().sort_index()

['Menores de un año', 'De un año', 'De 2', 'De 3', 'De 4', 'De 5 a 9', 'De 10 a 14', 'De 15 a 19', 'De 20 a 24', 'De 25 a 29', 'De 30 a 34', 'De 35 a 39', 'De 40 a 44', 'De 45 a 49', 'De 50 a 54', 'De 55 a 59', 'De 60 a 64', 'De 65 a 69', 'De 70 a 74', 'De 75 a 79', 'De 80 a 84', 'De 85 a 89', 'De 90 a 94', 'De 95 a 99', 'De 100 a 104', 'De 105 a 109', 'De 110 a 114', 'De 115 a 119', 'De 120 y más', 'No especificada']


EDAD_AGRU
Menores de un año     6739
De un año              606
De 2                   289
De 3                   210
De 4                   145
De 5 a 9               374
De 10 a 14             398
De 15 a 19             851
De 20 a 24            1667
De 25 a 29            2523
De 30 a 34            3221
De 35 a 39            3224
De 40 a 44            3521
De 45 a 49            3805
De 50 a 54            4454
De 55 a 59            5351
De 60 a 64            6625
De 65 a 69            8025
De 70 a 74            9409
De 75 a 79           10933
De 80 a 84           11383
De 85 a 89           10394
De 90 a 94            6738
De 95 a 99            2614
De 100 a 104           519
De 105 a 109            48
De 110 a 114             9
De 115 a 119             1
De 120 y más             1
No especificada        257
Name: count, dtype: int64

### Conversión de variables de fecha

Para utilizar las herramientas de análisis temporal de Pandas, es necesario combinar las columnas de día, mes y año en una sola columna con el tipo de dato `datetime`. Esto nos permitirá realizar operaciones de tiempo de manera eficiente, como agrupar los datos por mes o calcular la diferencia entre fechas.

Para lograr esto, usaremos la función `pd.to_datetime()`. Esta función es flexible y puede convertir múltiples columnas a un solo objeto de fecha y hora. 

Es importante que las columnas que se utilizan para la conversión tengan los nombres específicos `['year','month','day'.]`. Al renombrar las columnas del DataFrame temporal, se asegura que `pd.to_datetime()` interprete correctamente cada variable, lo que garantiza una conversión precisa y evita errores.

In [43]:
data = df_respiratorio[['ANIO_OCUR', 'MES_OCURR', 'DIA_OCURR']]
data.columns = ['year', 'month', 'day']

df_respiratorio['FECHA_OCURR'] = pd.to_datetime(data, errors='coerce')

df_respiratorio['FECHA_OCURR'].tail()

104329   2023-03-11
104330   2023-06-19
104331   2023-12-22
104332   2023-08-09
104333   2023-04-05
Name: FECHA_OCURR, dtype: datetime64[ns]

Con este paso, hemos completado la conversión de las variables clave. El DataFrame ahora tiene una columna de fecha que nos permitirá hacer análisis temporales. También hemos asegurado que las variables categóricas tengan el tipo correcto, lo que mejora el rendimiento del análisis.

In [ ]:
df_respiratorio.dtypes

## 4.f Ingeniería de variables

La **ingeniería de variables** consiste en crear nuevas variables a partir de las ya existentes para hacerlas más informativas. Este proceso transforma los datos sin procesar, haciéndolos más adecuados para el análisis o para ser utilizados en modelos de aprendizaje automático.

En este caso, aprovecharemos la columna `FECHA_OCURR` de tipo `datetime` para crear una variable que nos diga el día de la semana en que ocurrió la defunción. Esto nos puede ayudar a identificar patrones de mortalidad por día de la semana.

El proceso será:

1.  Extraer el día de la semana de la fecha.
2.  Mapear los números de los días a sus nombres en español.
3.  Convertir la nueva columna a un tipo de dato categórico ordenado.

In [44]:
# Extraer el día de la semana (0=Lunes, 6=Domingo)
df_respiratorio['DIA_SEMANA_OCURR'] = df_respiratorio['FECHA_OCURR'].dt.dayofweek

# Crear un diccionario para mapear el número a los nombres en español
mapeo_dias = { 0: 'Lunes', 1: 'Martes', 2: 'Miércoles', 3: 'Jueves', 4: 'Viernes', 5: 'Sábado', 6: 'Domingo'  }

# Crear la nueva columna con los nombres de los días
df_respiratorio['DIA_SEMANA_OCURR'] = df_respiratorio['DIA_SEMANA_OCURR'].replace(mapeo_dias)

# Definir el orden de los días de la semana
orden_dias = list(mapeo_dias.values())

# Convertir la columna a tipo categórico ordenado
df_respiratorio['DIA_SEMANA_OCURR'] = pd.Categorical(
    df_respiratorio['DIA_SEMANA_OCURR'],
    categories=orden_dias,
    ordered=True
)

# Verificar el cambio
df_respiratorio['DIA_SEMANA_OCURR'].tail()

104329       Sábado
104330        Lunes
104331      Viernes
104332    Miércoles
104333    Miércoles
Name: DIA_SEMANA_OCURR, dtype: category
Categories (7, object): ['Lunes' < 'Martes' < 'Miércoles' < 'Jueves' < 'Viernes' < 'Sábado' < 'Domingo']

Este tipo de procesos nos permite generar variables derivadas que facilitan el responder preguntas de investigación específicas, como en este caso, identificar patrones temporales en la mortalidad.

## 4.g Guardar el conjunto de datos limpio

Una vez que hemos completado la limpieza y preparación de los datos, el paso final es guardar el DataFrame resultante. Almacenar la versión limpia del archivo evita tener que repetir todo el proceso de limpieza cada vez que se desee realizar un nuevo análisis.

Para esto, usaremos dos formatos comunes: CSV y Pickle.

Un **archivo CSV** (Comma-Separated Values) es un formato de texto simple que almacena datos tabulares. Cada fila es un registro de datos y cada campo de un registro está separado por una coma. Tiene la ventaja de que es **universal y legible**. Puedes abrirlo con cualquier editor de texto o software de hojas de cálculo. Esto facilita la colaboración, ya que cualquier persona puede usarlo sin necesidad de software o librerías específicas. Sin embargo, **no conserva el tipo de dato de las columnas** ni la estructura del índice del DataFrame. Al volver a cargarlo, Pandas debe inferir los tipos de datos, lo que puede causar errores si la inferencia no es correcta.

El **formato Pickle de Python** serializa un objeto de Python, como un DataFrame de Pandas, en un formato binario. Esto significa que almacena el objeto tal como está, incluyendo sus tipos de datos, índice y estructura. Su principal ventaja es que es **eficiente y mantiene los tipos de datos de las columnas**. Al cargar un archivo Pickle, se obtiene un objeto idéntico al que se guardó. Esto es ideal para flujos de trabajo donde se requiere cargar y guardar datos sin perder la estructura ni el tipo de dato. Sin embargo, **no es legible ni portable**. Solo se puede leer con la librería `pickle` de Python, y su uso no es compatible con otros lenguajes o programas. Se debe tener precaución al abrir archivos Pickle de fuentes no confiables, ya que pueden ejecutar código malicioso.

In [45]:
# Guardar el DataFrame en formato CSV
df_respiratorio.to_csv('data_clean/defunciones_respiratorias_2023.csv', index=False)

# Guardar el DataFrame en formato Pickle
df_respiratorio.to_pickle('data_clean/defunciones_respiratorias_2023.pkl')

Has completado el proceso de limpieza de datos. Ahora tienes un conjunto de datos limpio que está listo para la etapa de análisis y visualización.

## 4.h Resumen

En esta lección hemos aprendido varios conceptos:
* Una buena pregunta de investigación es  centrada en un tema, específica, medible y relevante.
    * Además, debe ser factible de responder con los datos y recursos disponibles.
* Antes de empezar a limpiar los datos es necesario plantear una estrategia que incluya para cada variable/columna.
    * Seleccionar variables y observaciones relevantes.
    * Manejo de faltantes.
    * Limpieza necesaria por tipo de dato.
    * Validación de la limpieza.
* Aplicar funciones especiales con `.apply()`.
* Remplazar textos o parte del texto `.replace()`.
    * Se puede usar un diccionario de remplazo.
    * Unir varias tablas con `pd.merge()`.
* Convertir a categóricos con `.astype('category')`.
    * Fijar el catálogo válido.
    * Ingeniería de variables derivadas.
    * Ordenar el catálogo si es necesario.
* Convertir a tipo de fecha con `pd.to_datetime()`:
    * Se pueden obtener, año, mes, día, día de la semana, etc
* Guardar.
    * Es necesario validar la limpieza antes de guardar.
    * En formato `CSV` para compartir `.to_csv()`.
    * En formato `pickle` para analizar en Python `.to_pickle`.

**¡Gracias!**